In [1]:
import pandas as pd
import numpy as np

In [2]:
# 엑셀 파일에서 실제 시트명들을 가져옵니다
try:
    excel_file = pd.ExcelFile("1년치 취합.xlsx")
    all_sheets = excel_file.sheet_names
    print("파일에 있는 모든 시트:", all_sheets)
    
    # 202307부터 202507 범위의 시트만 필터링
    target_sheets = []
    for sheet in all_sheets:
        if len(sheet) == 6 and sheet.isdigit():
            if "202307" <= sheet <= "202507":
                target_sheets.append(sheet)
    
    target_sheets.sort()
    print("처리할 시트들:", target_sheets)
    
    if not target_sheets:
        print("처리할 시트가 없습니다. 시트명을 확인해주세요.")
        exit()
        
except FileNotFoundError:
    print("파일을 찾을 수 없습니다: 1년치 취합.xlsx")
    exit()
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    exit()

# 각 시트 처리
for sheet_name in target_sheets:
    print(f"\n처리 중: {sheet_name}")
    
    try:
        # 데이터 읽기
        df = pd.read_excel(excel_file, sheet_name=sheet_name, header=None)
        
        # 첫 2행 제거 후 인덱스 리셋
        df_drop = df.drop(index=df.index[:2])
        df_origin = df_drop.reset_index(drop=True)
        
        # 컬럼명 설정
        df_origin.columns = ["Category1"] + df_origin.iloc[0, 1:].tolist()
        df_origin = df_origin.drop(df_origin.index[0])
        
        # 홀수열 이름 복사
        column_list = list(df_origin.columns)
        for j in range(1, len(column_list), 2):
            if j+1 < len(column_list):
                column_list[j+1] = column_list[j]
        df_origin.columns = column_list
        
        # 전체가 비어있는 행 처리
        all_null_rows = df_origin.isnull().all(axis=1)
        try:
            first_null_row_index = all_null_rows.idxmax()
            df_result = df_origin.iloc[:first_null_row_index]
        except ValueError:
            df_result = df_origin
        
        # null 컬럼 찾기
        is_all_null = df_result.isnull().all()
        try:
            first_null_col_index = is_all_null.tolist().index(True)
        except ValueError:
            first_null_col_index = len(df_result.columns)  # null 컬럼 없음 처리
        
        # 채권/주식 분리
        df_채권 = df_result.iloc[:, :first_null_col_index]
        df_주식_temp = df_result.iloc[:, first_null_col_index:]
        
        # 주식 데이터 시작 열 찾기
        is_data_present = ~df_주식_temp.isnull().all()
        try:
            first_data_col_index = is_data_present.tolist().index(True)
            df_주식 = df_주식_temp.iloc[:, first_data_col_index:]
        except ValueError:
            df_주식 = pd.DataFrame()
        
        # 주식 첫 컬럼 이름 변경
        if not df_주식.empty:
            df_주식 = df_주식.rename(columns={df_주식.columns[0]: 'Category2'})
        
        # globals()로 시트명 기반 변수 생성
        globals()[f'df{sheet_name}'] = df_result
        globals()[f'df{sheet_name}_채권'] = df_채권
        globals()[f'df{sheet_name}_주식'] = df_주식
        
        print(f"{sheet_name}, {sheet_name}_채권, {sheet_name}_주식 생성 완료")
        print(f"df{sheet_name} 크기: {df_result.shape}")
        print(f"df{sheet_name}_채권 크기: {df_채권.shape}")
        print(f"df{sheet_name}_주식 크기: {df_주식.shape}")
        
    except Exception as e:
        print(f"시트 {sheet_name} 처리 중 오류 발생: {e}")
        continue

print("\n모든 시트 처리 완료!")

# 생성된 변수 확인
created_dfs = [var for var in globals().keys() if var.startswith('df202')]
created_dfs.sort()
print("생성된 데이터프레임:", created_dfs)


파일에 있는 모든 시트: ['Sheet1', '데이터', '202307', '202308', '202309', '202310', '202311', '202312', '202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408', '202409', '202410', '202411', '202412', '202501', '202502', '202503', '202504', '202505', '202506']
처리할 시트들: ['202307', '202308', '202309', '202310', '202311', '202312', '202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408', '202409', '202410', '202411', '202412', '202501', '202502', '202503', '202504', '202505', '202506']

처리 중: 202307
202307, 202307_채권, 202307_주식 생성 완료
df202307 크기: (43, 35)
df202307_채권 크기: (43, 9)
df202307_주식 크기: (43, 23)

처리 중: 202308
202308, 202308_채권, 202308_주식 생성 완료
df202308 크기: (43, 35)
df202308_채권 크기: (43, 9)
df202308_주식 크기: (43, 23)

처리 중: 202309
202309, 202309_채권, 202309_주식 생성 완료
df202309 크기: (43, 35)
df202309_채권 크기: (43, 9)
df202309_주식 크기: (43, 23)

처리 중: 202310
202310, 202310_채권, 202310_주식 생성 완료
df202310 크기: (43, 55)
df202310_채권 크기: (43, 9)
df202310_주식 크기: (43, 43

In [3]:
# 1. 채권형

In [4]:
# 처리할 채권/주식 데이터프레임 목록 가져오기
채권_dfs = [name for name in globals() 
            if name.startswith('df20') and name.endswith('_채권') 
            and len(name.split('_')[0]) == 8]  # df202307
채권_dfs.sort()

주식_dfs = [name for name in globals() 
            if name.startswith('df20') and name.endswith('_주식') 
            and len(name.split('_')[0]) == 8]  # df202307
주식_dfs.sort()

# 채권형 최종 데이터 생성
for df_name in 채권_dfs:
    df_채권 = globals()[df_name]
    
    category = df_채권.iloc[:, 0]
    result = []

    for i in range(1, df_채권.shape[1], 2):
        left_col = df_채권.columns[i]
        right_col = df_채권.columns[i+1]
        fund_code = left_col

        for idx, cat in enumerate(category):
            if isinstance(cat, str):
                result.append({
                    "펀드코드": fund_code,
                    "데이터ID(DATA_ID)": cat,
                    "비중(WGT)": df_채권.iloc[idx, i+1]
                })
            else:
                result.append({
                    "펀드코드": fund_code,
                    "데이터ID(DATA_ID)": df_채권.iloc[idx, i+1],
                    "비중(WGT)": df_채권.iloc[idx, i]
                })

    new_df_채권 = pd.DataFrame(result)
    new_df_채권 = new_df_채권.dropna(subset=['비중(WGT)'])
    
    # Mapping
    category_map = {
        '0-3Y': 'Fixed Income_만기',
        '3-5Y': 'Fixed Income_만기',
        '5-10Y': 'Fixed Income_만기',
        '>10Y': 'Fixed Income_만기',
        'AAA': 'Fixed Income_신용등급',
        'AA': 'Fixed Income_신용등급',
        'A': 'Fixed Income_신용등급',
        'BBB': 'Fixed Income_신용등급',
        '<BBB': 'Fixed Income_신용등급',
        'NR': 'Fixed Income_신용등급',
        
        'Treasury': 'Fixed Income_섹터',
        'Government': 'Fixed Income_섹터',
        'Corporate': 'Fixed Income_섹터',
        'Securitized': 'Fixed Income_섹터',
        
        'Developed': 'Fixed Income_Region1',
        'Emerging': 'Fixed Income_Region1',
        
        'North America': 'Fixed Income_Region2',
        'Europe': 'Fixed Income_Region2',
        'Japan': 'Fixed Income_Region2',
        'Asia Pacific': 'Fixed Income_Region2',
        'Latin America': 'Fixed Income_Region2',
        'Africa/Middle East': 'Fixed Income_Region2',
        'South Africa' : 'Fixed Income_Region2',
        
        'United States': 'Fixed Income_Country',
        'United Kingdom': 'Fixed Income_Country',
        'United Arab Emirates': 'Fixed Income_Country',
        'Japan': 'Fixed Income_Country',
        'China': 'Fixed Income_Country',
        'Canada': 'Fixed Income_Country',
        'Italy': 'Fixed Income_Country',
        'Australia': 'Fixed Income_Country',
        'South Korea': 'Fixed Income_Country',
        'France': 'Fixed Income_Country',
        'Supranational': 'Fixed Income_Country',
        'Switzerland': 'Fixed Income_Country',
        'Mexico': 'Fixed Income_Country',
        'Netherlands': 'Fixed Income_Country',
        'Spain': 'Fixed Income_Country',
        'Ireland': 'Fixed Income_Country',
        'Denmark': 'Fixed Income_Country',
        'New Zealand': 'Fixed Income_Country',
        'Indonesia': 'Fixed Income_Country',
        'Saudi Arabia': 'Fixed Income_Country',
        'Brazil': 'Fixed Income_Country',
        'Germany': 'Fixed Income_Country',
        'Belgium': 'Fixed Income_Country',
        'Romania': 'Fixed Income_Country',
        'Austria': 'Fixed Income_Country',
        'Finland': 'Fixed Income_Country',
        'Luxemburg': 'Fixed Income_Country',
        'EU': 'Fixed Income_Country',
        'Greece': 'Fixed Income_Country',
        'Hungary': 'Fixed Income_Country',
        'India': 'Fixed Income_Country',
        'Israel': 'Fixed Income_Country',
        'Latvia': 'Fixed Income_Country',
        'Poland': 'Fixed Income_Country',
        'Portugal': 'Fixed Income_Country',
        'Slovakia': 'Fixed Income_Country',
        'South Africa': 'Fixed Income_Country',
        'Taiwan': 'Fixed Income_Country',
        'Sweden': 'Fixed Income_Country',
        'Hong Kong': 'Fixed Income_Country',
        'Peru': 'Fixed Income_Country',
        'Uruguay': 'Fixed Income_Country',
        'Norway': 'Fixed Income_Country',
        'Korea': 'Fixed Income_Country',
        'Singapore': 'Fixed Income_Country',
        'Bermuda': 'Fixed Income_Country',
        'Poland': 'Fixed Income_Country',
        'Israel': 'Fixed Income_Country',
        'Luxembourg': 'Fixed Income_Country',
        'Greece': 'Fixed Income_Country',
        'Chile': 'Fixed Income_Country',
        'Colombia': 'Fixed Income_Country',
        'Iceland': 'Fixed Income_Country',
        'Cayman Islands': 'Fixed Income_Country',
        'Supranationals': 'Fixed Income_Country',
        'Croatia': 'Fixed Income_Country',
        'Thailand': 'Fixed Income_Country',
        'Philippines': 'Fixed Income_Country',
        'Mauritius': 'Fixed Income_Country',
        'Egypt': 'Fixed Income_Country',
        'Argentina': 'Fixed Income_Country',
        'Malaysia': 'Fixed Income_Country',
        'Kazakhstan': 'Fixed Income_Country',
        'Slovenia': 'Fixed Income_Country',
        'Vietnam': 'Fixed Income_Country',
        'Ghana': 'Fixed Income_Country',
        'Estonia': 'Fixed Income_Country',
        'Czech Republic': 'Fixed Income_Country',
        'S.Korea': 'Fixed Income_Country',
    }

    new_df_채권['자산구분(ASSET_ID)'] = new_df_채권['데이터ID(DATA_ID)'].map(category_map)
    new_df_채권['자산구분(ASSET_ID)'] = new_df_채권['자산구분(ASSET_ID)'].fillna('기타_분류 없음')

    # 최종 컬럼 추가
    final_df = new_df_채권.copy()
    기준일 = df_name[2:8] + '01'  # df202307_채권 -> 20230701
    final_df['기준일'] = 기준일
    final_df['구분'] = 'FUND'
    final_df['기관코드'] = '1061'
    final_df['BM코드'] = ''
    final_df['데이터구분'] = 'FUND'

    final_columns = ['기준일','구분','기관코드','펀드코드','BM코드','자산구분(ASSET_ID)','데이터ID(DATA_ID)','데이터구분','비중(WGT)']
    final_df = final_df.reindex(columns=final_columns)

    # globals()에 최종 데이터프레임 저장
    globals()[f"final_{df_name}"] = final_df

    print(f"{df_name} -> final_{df_name} 생성 완료 (크기: {final_df.shape})")


df202307_채권 -> final_df202307_채권 생성 완료 (크기: (168, 9))
df202308_채권 -> final_df202308_채권 생성 완료 (크기: (168, 9))
df202309_채권 -> final_df202309_채권 생성 완료 (크기: (168, 9))
df202310_채권 -> final_df202310_채권 생성 완료 (크기: (168, 9))
df202311_채권 -> final_df202311_채권 생성 완료 (크기: (168, 9))
df202312_채권 -> final_df202312_채권 생성 완료 (크기: (168, 9))
df202401_채권 -> final_df202401_채권 생성 완료 (크기: (168, 9))
df202402_채권 -> final_df202402_채권 생성 완료 (크기: (168, 9))
df202403_채권 -> final_df202403_채권 생성 완료 (크기: (168, 9))
df202404_채권 -> final_df202404_채권 생성 완료 (크기: (168, 9))
df202405_채권 -> final_df202405_채권 생성 완료 (크기: (168, 9))
df202406_채권 -> final_df202406_채권 생성 완료 (크기: (168, 9))
df202407_채권 -> final_df202407_채권 생성 완료 (크기: (168, 9))
df202408_채권 -> final_df202408_채권 생성 완료 (크기: (168, 9))
df202409_채권 -> final_df202409_채권 생성 완료 (크기: (168, 9))
df202410_채권 -> final_df202410_채권 생성 완료 (크기: (168, 9))
df202411_채권 -> final_df202411_채권 생성 완료 (크기: (294, 9))
df202412_채권 -> final_df202412_채권 생성 완료 (크기: (294, 9))
df202501_채권 -> final_df20250

In [5]:
# final_로 시작하고 _채권으로 끝나는 DataFrame 이름 찾기
final_df_names_채권 = [name for name in globals() 
                  if name.startswith('final_df20') and name.endswith('_채권')]

final_df_names_채권.sort()  # 기준일 순서로 정렬

# 모든 final_df 불러와서 리스트에 담기
final_dfs_채권 = [globals()[name] for name in final_df_names_채권]

# 행 기준으로 합치기
merged_df_채권 = pd.concat(final_dfs_채권, ignore_index=True)

print(f"병합 완료: {merged_df_채권.shape}")

# 필요하면 globals()에 저장
globals()['merged_final_채권'] = merged_df_채권


병합 완료: (5040, 9)


In [6]:
# 'Japan'인 행만 마스크
mask = merged_df_채권['데이터ID(DATA_ID)'] == 'Japan'

# 펀드별 + 일자별로 묶어서 각 그룹의 첫 번째 행 인덱스만 뽑기
idx = (
    merged_df_채권[mask]
    .groupby(['펀드코드', '기준일'])
    .apply(lambda g: g.index.min())
)

# 첫 번째 행의 '자산구분(ASSET_ID)' 컬럼 값 변경
merged_df_채권.loc[idx.values, '자산구분(ASSET_ID)'] = 'Fixed Income_Region2'

C:\Users\user\AppData\Local\Temp\ipykernel_13040\4239884265.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.index.min())


In [7]:
# 2. 주식형

In [8]:
# 처리할 주식형 데이터프레임 목록 가져오기
주식_dfs = [name for name in globals() 
            if name.startswith('df20') and name.endswith('_주식') 
            and len(name.split('_')[0]) == 8]  # df202307_주식 등
주식_dfs.sort()

# 주식형 최종 데이터 생성
for df_name in 주식_dfs:
    df_주식 = globals()[df_name]
    
    # 첫 번째 열은 Category2
    category = df_주식.iloc[:, 0]
    result = []

    # 2열씩 묶어서 처리
    for i in range(1, df_주식.shape[1], 2):
        left_col = df_주식.columns[i]
        right_col = df_주식.columns[i+1]
        fund_code = left_col

        for idx, cat in enumerate(category):
            if isinstance(cat, str):
                result.append({
                    "펀드코드": fund_code,
                    "데이터ID(DATA_ID)": cat,
                    "비중(WGT)": df_주식.iloc[idx, i+1]
                })
            else:
                result.append({
                    "펀드코드": fund_code,
                    "데이터ID(DATA_ID)": df_주식.iloc[idx, i+1],
                    "비중(WGT)": df_주식.iloc[idx, i]
                })

    # 데이터프레임 생성
    new_df_주식 = pd.DataFrame(result)

    # 비중(WGT) NaN 제거
    new_df_주식 = new_df_주식.dropna(subset=['비중(WGT)'])
    # 비중(WGT)가 0인 행 제거 (문자열으로 들어가 있을 경우'0'도 제거)
    new_df_주식 = new_df_주식[new_df_주식['비중(WGT)'] != 0]
    new_df_주식 = new_df_주식[new_df_주식['비중(WGT)'] != '0']

    # Mapping
    category_map_주식 = {
        'Consumer Discretionary': 'Equity_Sector',
        'Consumer Staples': 'Equity_Sector',
        'Energy': 'Equity_Sector',
        'Financials': 'Equity_Sector',
        'Health Care': 'Equity_Sector',
        'Industrials': 'Equity_Sector',
        'IT': 'Equity_Sector',
        'Materials': 'Equity_Sector',
        'Communication Services': 'Equity_Sector',
        'Utilities': 'Equity_Sector',
        'Real Estate': 'Equity_Sector',
        'Consumer Disc.': 'Equity_Sector',
        'Consumer Stap.': 'Equity_Sector',
        # 'Other': 'Equity_Sector',
        'Telco': 'Equity_Sector',
        
        'Developed': 'Equity_Region1',
        'Emerging': 'Equity_Region1',
               
        'Africa/Middle East': 'Equity_Region2',
        'Asia Pacific': 'Equity_Region2',
        'Europe': 'Equity_Region2',
        'Latin America': 'Equity_Region2',
        'North America': 'Equity_Region2',

        'USA': 'Equity_Country',
        'United States': 'Equity_Country',
        'United Kingdom': 'Equity_Country',
        'United Arab Emirates': 'Equity_Country',
        'United Arab Emirate': 'Equity_Country',
        'Japan': 'Equity_Country',
        'China': 'Equity_Country',
        'Canada': 'Equity_Country',
        'Italy': 'Equity_Country',
        'Australia': 'Equity_Country',
        'South Korea': 'Equity_Country',
        'France': 'Equity_Country',
        'Supranational': 'Equity_Country',
        'Switzerland': 'Equity_Country',
        'Mexico': 'Equity_Country',
        'Netherlands': 'Equity_Country',
        'Spain': 'Equity_Country',
        'Ireland': 'Equity_Country',
        'Denmark': 'Equity_Country',
        'New Zealand': 'Equity_Country',
        'Indonesia': 'Equity_Country',
        'Saudi Arabia': 'Equity_Country',
        'Brazil': 'Equity_Country',
        'Germany': 'Equity_Country',
        'Belgium': 'Equity_Country',
        'Romania': 'Equity_Country',
        'Austria': 'Equity_Country',
        'Finland': 'Equity_Country',
        'Luxemburg': 'Equity_Country',
        'Luxembourg': 'Equity_Country',
        'EU': 'Equity_Country',
        'Greece': 'Equity_Country',
        'Hungary': 'Equity_Country',
        'India': 'Equity_Country',
        'Israel': 'Equity_Country',
        'Latvia': 'Equity_Country',
        'Poland': 'Equity_Country',
        'Portugal': 'Equity_Country',
        'Slovakia': 'Equity_Country',
        'South Africa': 'Equity_Country',
        'Taiwan': 'Equity_Country',
        'Sweden': 'Equity_Country',
        'Hong Kong': 'Equity_Country',
        'Peru': 'Equity_Country',
        'Uruguay': 'Equity_Country',
        'Norway': 'Equity_Country',
        'Korea': 'Equity_Country',
        'Singapore': 'Equity_Country',
        'Bermuda': 'Equity_Country',
        'Poland': 'Equity_Country',
        'Israel': 'Equity_Country',
        'Luxembourg': 'Equity_Country',
        'Greece': 'Equity_Country',
        'Chile': 'Equity_Country',
        'Colombia': 'Equity_Country',
        'Iceland': 'Equity_Country',
        'Cayman Islands': 'Equity_Country',
        'Supranationals': 'Equity_Country',
        'Croatia': 'Equity_Country',
        'Thailand': 'Equity_Country',
        'Philippines': 'Equity_Country',
        'Mauritius': 'Equity_Country',
        'Egypt': 'Equity_Country',
        'Argentina': 'Equity_Country',
        'Malaysia': 'Equity_Country',
        'Kazakhstan': 'Equity_Country',
        'Slovenia': 'Equity_Country',
        'Vietnam': 'Equity_Country',
        'Ghana': 'Equity_Country', 
        'Netherland': 'Equity_Country',
        'Columbia': 'Equity_Country',
        'Pakistan': 'Equity_Country',
        'Zambia': 'Equity_Country',
        'Saudi Arabia  ': 'Equity_Country',
        'Saudi Arabia': 'Equity_Country',
        'Burkina Faso': 'Equity_Country',
        'Cash': 'Equity_Country', ## 삭제
        'Other (Cash)': 'Equity_Country' ## 삭제
    }

    new_df_주식['자산구분(ASSET_ID)'] = new_df_주식['데이터ID(DATA_ID)'].map(category_map_주식)
    new_df_주식['자산구분(ASSET_ID)'] = new_df_주식['자산구분(ASSET_ID)'].fillna('기타_분류 없음')

    # 최종 컬럼 추가
    final_df = new_df_주식.copy()
    기준일 = df_name[2:8] + '01'  # df202307_주식 -> 20230701
    final_df['기준일'] = 기준일
    final_df['구분'] = 'FUND'
    final_df['기관코드'] = '1061'
    final_df['BM코드'] = ''
    final_df['데이터구분'] = 'FUND'

    final_columns = ['기준일','구분','기관코드','펀드코드','BM코드','자산구분(ASSET_ID)','데이터ID(DATA_ID)','데이터구분','비중(WGT)']
    final_df = final_df.reindex(columns=final_columns)

    # globals()에 최종 데이터프레임 저장
    globals()[f"final_{df_name}"] = final_df

    print(f"{df_name} -> final_{df_name} 생성 완료 (크기: {final_df.shape})")


df202307_주식 -> final_df202307_주식 생성 완료 (크기: (360, 9))
df202308_주식 -> final_df202308_주식 생성 완료 (크기: (348, 9))
df202309_주식 -> final_df202309_주식 생성 완료 (크기: (352, 9))
df202310_주식 -> final_df202310_주식 생성 완료 (크기: (595, 9))
df202311_주식 -> final_df202311_주식 생성 완료 (크기: (592, 9))
df202312_주식 -> final_df202312_주식 생성 완료 (크기: (589, 9))
df202401_주식 -> final_df202401_주식 생성 완료 (크기: (823, 9))
df202402_주식 -> final_df202402_주식 생성 완료 (크기: (852, 9))
df202403_주식 -> final_df202403_주식 생성 완료 (크기: (973, 9))
df202404_주식 -> final_df202404_주식 생성 완료 (크기: (1052, 9))
df202405_주식 -> final_df202405_주식 생성 완료 (크기: (1019, 9))
df202406_주식 -> final_df202406_주식 생성 완료 (크기: (1017, 9))
df202407_주식 -> final_df202407_주식 생성 완료 (크기: (1010, 9))
df202408_주식 -> final_df202408_주식 생성 완료 (크기: (1013, 9))
df202409_주식 -> final_df202409_주식 생성 완료 (크기: (1002, 9))
df202410_주식 -> final_df202410_주식 생성 완료 (크기: (1001, 9))
df202411_주식 -> final_df202411_주식 생성 완료 (크기: (1006, 9))
df202412_주식 -> final_df202412_주식 생성 완료 (크기: (997, 9))
df202501_주식 -> final

In [9]:
# final_로 시작하고 _주식으로 끝나는 DataFrame 이름 찾기
final_df_names_주식 = [name for name in globals() 
                  if name.startswith('final_df20') and name.endswith('_주식')]

final_df_names_주식.sort()  # 기준일 순서로 정렬

# 모든 final_df 불러와서 리스트에 담기
final_dfs_주식 = [globals()[name] for name in final_df_names_주식]

# 행 기준으로 합치기
merged_df_주식 = pd.concat(final_dfs_주식, ignore_index=True)

print(f"병합 완료: {merged_df_주식.shape}")

# 필요하면 globals()에 저장
globals()['merged_final_주식'] = merged_df_주식

병합 완료: (20713, 9)


In [10]:
# 완전 동일한 행(모든 컬럼 동일) 중 중복된 행만 보기
dupes = merged_df_주식[merged_df_주식.duplicated(keep=False)]

print("중복된 행(모든 컬럼 동일):")
print(dupes)
print(f"중복된 행 개수: {len(dupes)}")

중복된 행(모든 컬럼 동일):
            기준일    구분  기관코드          펀드코드 BM코드  자산구분(ASSET_ID) 데이터ID(DATA_ID)  \
231    20230701  FUND  1061  KRL232106112       Equity_Country          Japan   
235    20230701  FUND  1061  KRL232106112       Equity_Country          Japan   
267    20230701  FUND  1061  KRL232106109       Equity_Country          Japan   
272    20230701  FUND  1061  KRL232106109       Equity_Country          Japan   
316    20230701  FUND  1061  KRL232106111       Equity_Country          Japan   
...         ...   ...   ...           ...  ...             ...            ...   
20583  20250601  FUND  1061  KRL203106108       Equity_Country         Taiwan   
20584  20250601  FUND  1061  KRL203106108       Equity_Country   South Africa   
20585  20250601  FUND  1061  KRL203106108       Equity_Country        Ireland   
20598  20250601  FUND  1061  KRL301106119       Equity_Country          Japan   
20604  20250601  FUND  1061  KRL301106119       Equity_Country          Japan   

      데이터구

In [11]:
# 모든 컬럼이 완전히 같은 행만 제거
duplicates_df_주식 = merged_df_주식.drop_duplicates()

In [12]:
# 중복 제거 전 행 수
before = len(merged_df_주식)

# 중복 제거
duplicates_df_주식 = merged_df_주식.drop_duplicates()

# 중복 제거 후 행 수
after = len(duplicates_df_주식)

print(f"삭제된 행 개수: {before - after}")

삭제된 행 개수: 673


In [13]:
duplicates_df_주식.to_excel("duplicates_df_주식.xlsx", index = False, header = True)

In [ ]:
copy_duplicates_df_주식 = pd.read_excel("duplicates_df_주식.xlsx")

In [ ]:
copy_duplicates_df_주식

In [ ]:
# 완전히 같은 행 전부 보기
copy_duplicates_df_주식[copy_duplicates_df_주식.duplicated(keep=False)]

In [ ]:
# 모든 컬럼이 완전히 같은 행만 제거
de_duplicates_df_주식 = copy_duplicates_df_주식.drop_duplicates()

In [ ]:
# 중복 제거 전 행 수
before = len(copy_duplicates_df_주식)

# 중복 제거
de_duplicates_df_주식 = copy_duplicates_df_주식.drop_duplicates()

# 중복 제거 후 행 수
after = len(de_duplicates_df_주식)

print(f"삭제된 행 개수: {before - after}")

In [ ]:
de_duplicates_df_주식.to_excel("de_duplicates_df_주식.xlsx", index = False, header = True)

In [24]:
# japan개수가 하나일 경우 
mask1_df = de_duplicates_df_주식.copy()

# (펀드코드, 기준일)로 그룹핑해서 Japan이 몇 개인지 확인
mask1 = (
    mask1_df['데이터ID(DATA_ID)'].eq('Japan') &
    mask1_df.groupby(['펀드코드','기준일'])['데이터ID(DATA_ID)'].transform(
        lambda x: (x=='Japan').sum() == 1
    )
)

# 조건에 맞는 행만 복제
rows_to_duplicate = mask1_df[mask1]

# 원본 df에 행 추가(인덱스 리셋)
mask1_df_주식 = pd.concat([mask1_df, rows_to_duplicate], ignore_index=True)

In [29]:
rows_to_duplicate.to_excel("rows_to_duplicate.xlsx", index =False, header = True)

In [26]:
mask2 = mask1_df_주식['데이터ID(DATA_ID)'] == 'Japan'

idx = (
    mask1_df_주식[mask2]
    .groupby(['펀드코드', '기준일'])['데이터ID(DATA_ID)']  # 특정 컬럼 지정
    .apply(lambda g: g.index.min())
)

mask1_df_주식.loc[idx.values, '자산구분(ASSET_ID)'] = 'Equity_Region2'

In [30]:
mask1_df_주식 = mask1_df_주식.sort_index().reset_index(drop=True)

In [33]:
# 정렬
mask1_df_주식 = (
    mask1_df_주식
    .sort_values(['기준일', '펀드코드', '자산구분(ASSET_ID)'])
    .reset_index(drop=True)
)

In [37]:
# 'Japan'인 행만 중복 체크
mask_japan = mask1_df_주식['데이터ID(DATA_ID)'] == 'Japan'

# 동일한 (기준일,펀드코드,자산구분) 조합에서 'Japan'이 여러 개일 때
# 첫 번째는 제거(권역이랑 비중 동일), 그 이후 것들을 True로 표시
dup_idx = (
    mask1_df_주식[mask_japan]
    .duplicated(subset=['기준일','펀드코드','자산구분(ASSET_ID)'], keep = 'last')
)

# 원본 인덱스로 복원
idx_to_drop = mask1_df_주식[mask_japan].index[dup_idx]

# 해당 행 삭제
mask2_df_주식 = mask1_df_주식.drop(idx_to_drop).reset_index(drop=True)


In [38]:
mask2_df_주식.to_excel("mask2_df_주식.xlsx", index = False, header = True)

In [ ]:
# 3. 최종

In [39]:
# 두 개 병합 (행 기준)
merged_all_final = pd.concat([merged_df_채권, mask2_df_주식], ignore_index=True)

print(f"주식 + 채권 병합 완료: {merged_all_final.shape}")

# globals()에 저장
globals()['merged_final_주식_채권'] = merged_all_final

주식 + 채권 병합 완료: (25169, 9)


In [41]:
merged_all_final.to_excel("merged_all_final.xlsx", index = False, header = True)

In [50]:
# 기준일, 펀드코드별로 Japan 개수 세기
japan_counts = (
    merged_all_final[merged_all_final['데이터ID(DATA_ID)'] == 'Japan']  # DATA_ID가 Japan인 행만 선택
    .groupby(['기준일', '펀드코드'])                         # 기준일과 펀드코드로 그룹화
    .size()                                                 # 그룹별 행 개수 계산
    .reset_index(name='Japan_count')                        # 결과를 DataFrame으로 변환
)

display(
    japan_counts.style.set_table_attributes('style="max-height:400px; overflow-y: auto; display:block;"'))


,기준일,펀드코드,Japan_count
0,20230701,KRL107106105,2
1,20230701,KRL203106101,2
2,20230701,KRL203106106,2
3,20230701,KRL213106111,2
4,20230701,KRL223106106,2
5,20230701,KRL232106108,2
6,20230701,KRL232106109,2
7,20230701,KRL232106110,2
8,20230701,KRL232106111,2
9,20230701,KRL232106112,2


In [1]:
# Japan이 1개인 그룹 확인
japan_one = japan_counts[japan_counts['Japan_count'] == 1]

japan_one

NameError: name 'japan_counts' is not defined

### KRL301106121는 region2에 비중 -로 들어있음 : REPO 투자라서 운용사 실제 (-) 투자 

### => 5개 펀드는 비중 엑셀에서 직접 추가하기

In [20]:
import json

# 노트북 파일 경로
notebook_path = r"C:\Users\user\Desktop\업무\펀드 취합\펀드 취합_3.ipynb"
output_txt_path = r"C:\Users\user\Desktop\업무\펀드 취합\펀드 취합_code.txt"

# 노트북 읽기
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = json.load(f)

# 코드 셀만 추출
code_cells = [cell['source'] for cell in nb['cells'] if cell['cell_type'] == 'code']

# 텍스트 파일로 저장
with open(output_txt_path, "w", encoding="utf-8") as f:
    for cell in code_cells:
        f.write("".join(cell))
        f.write("\n\n# --- Cell Separator ---\n\n")  # 셀 구분 표시
